## Дополнительная задача №4

##### Автор: [Радослав Нейчев](https://www.linkedin.com/in/radoslav-neychev/), @neychev

Данная задача – упрощенный вариант первого домашнего задания.

В данном задании Вам необходимо пройти все этапы решения задачи на другой выборке. Ваша основная цель – проанализировать данные и сделать выводы на основе данных. Код для этого уже написан, но вы можете внести в него изменения по желанию.

## Работа с данными
Рассмотрим табличные [данные о качестве вина](https://archive.ics.uci.edu/ml/datasets/wine+quality). Каждый тип вина – отдельный объект, который описывается числовыми признаками и относится к одному из двух классов. Т.е. решается задача бинарной классификации, где классы соответствуют хорошему и плохому вину.

На что стоит обратить внимание при начале работы с табличными данными?
* На наличие пропусков в данных.
* На баланс/дисбаланс классов.
* На типы данных, с которыми предстоит работать.

In [ ]:
!pip install scikit-plot
!pip install plotly

In [ ]:
import sys
import json

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcParams.update({'font.size': 15})

from IPython.display import clear_output

print(f'Python version: {sys.version}\n')
import plotly.io as pio
import plotly.express as px

from plotly import graph_objects as go
import seaborn as sns
sns.set(style="ticks", context="talk")



In [ ]:
import pandas as pd

In [ ]:
file_link = 'https://raw.githubusercontent.com/girafe-ai/ml-course/23f_ptml/homeworks/hw_extra/white_wine_preprocessed.csv'

In [ ]:
dataset = pd.read_csv(file_link, index_col=0)

Датасет небольших размеров:

In [ ]:
dataset.shape

Оценим данные визуально:

In [ ]:
dataset.head()

Не слишком информативно. Рассмотрим статистики по всему датасету.

In [ ]:
dataset.describe()

In [ ]:
dataset.info()

In [ ]:
dataset.nunique()

Пропусков в данных нет, все значения признаков – рациональные числа.

In [ ]:
class_counts = dataset['quality_class'].value_counts()

In [ ]:
class_counts

In [ ]:
color_mapper = {'good': 'green', 'bad': 'blue', 'ok': 'orange'}

In [ ]:
plt.figure(figsize=(10, 10))
plt.pie(class_counts.values, labels=class_counts.index, autopct = '%.2f%%', colors=color_mapper.values())
plt.title('Different classes', fontsize = 30)



Хорошо ли сбалансированы классы, или один из классов значительно превосходит другие?

## Разведочный анализ и визуализация данных.
Качественная визуализация позволяет значительно ускорить формулирование и проверку гипотез. Есть множество способов представить многомерные данные, обратимся к наиболее простым и визуально информативным.

In [ ]:
feature_columns = dataset.columns[:-1]
target_column = dataset.columns[-1]

In [ ]:
dataset.quality_class

In [ ]:
target_column

In [ ]:
corr = dataset[feature_columns].corr()

heat = go.Heatmap(z=corr,
                  x=feature_columns,
                  y=feature_columns,
                  xgap=1, ygap=1,
                  colorbar_thickness=20,
                  colorbar_ticklen=3
                   )

layout = go.Layout(title_text="Correlation matrix", title_x=0.5, 
                   width=600, height=600,
                   xaxis_showgrid=False,
                   yaxis_showgrid=False,
                   yaxis_autorange='reversed')
   
fig=go.Figure(data=[heat], layout=layout)   
fig.update_layout(
    xaxis = dict(
        tickmode = 'linear',
        tick0 = 0,
        dtick = 1
    ),
    yaxis = dict(
        tickmode = 'linear',
        tick0 = 0,
        dtick = 1
    )
)
fig.show()

Наблюдается ли сильная корреляция между признаками? Стоит ли исключить некоторые из них? Чтобы сохранить все признаки, оставьте список пустым (т.е. просто `[]`)

In [ ]:
# Чтобы исключить признаки, укажите их номера (начиная с 0) в списке ниже
# например, чтобы исключить признаки с номерами 4 и 7:
# _drop_features_indices = [4, 7]
_drop_features_indices = []
print(feature_columns[_drop_features_indices])
selected_feature_columns = [feature for idx, feature in enumerate(feature_columns) if idx not in _drop_features_indices]

Оставшиеся признаки доступны ниже.

In [ ]:
selected_feature_columns

Отобразим объекты в двумерное пространство с помощью t-SNE. Это вероятностная техника снижения размерности, часто используемая для визуализации. Этот метод старается сохранить близкие объекты близко друг к другу, а далекие – далеко друг от друга в пространстве меньшей размерности. Более подробно с t-SNE можно ознакомиться [здесь](https://habr.com/ru/post/267041/), обзор методов по ускорению данной техники доступно [здесь](https://habr.com/ru/post/341208/). 

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
from sklearn.manifold import TSNE
tsne = TSNE()
X_transformed = tsne.fit_transform(StandardScaler().fit_transform(dataset[selected_feature_columns].values))
# mapper = {x: idx for idx, x in enumerate(list(set(dataset[target_column])))}
plt.figure(figsize=(12, 10))
for class_label in np.unique(dataset[target_column].values):
    ix = np.where(dataset[target_column].values==class_label)
    plt.scatter(X_transformed[ix,0], X_transformed[ix, 1], c=color_mapper[class_label], label=class_label)
plt.legend()

Что можно сказать о данных на основе подобной визуализации? Можете ли вы сформулировать какие-либо гипотезы на основе данной картины?

__Гипотезы:__

## Обзор основных мер качества в задаче классификации
К сожалению, для оценки качества итоговой модели далеко не всегда удается учесть ее прямое влияние на бизнес. В таких случаях стоит использовать те критерии, которые наиболее согласованы с бизнес значимостью решения. Далее рассмотрим основные способы оценки качества в задачах классификации.

__Accuracy__ определяет долю правильно предсказанных меток класса к общему числу объектов. Используется почти всегда вместе с другими метриками, но не подходит для случая сильно несбалансированных классов. В таких случаях может использоваться balanced accuracy.

__Precision или же Точность__ требует выбора целевого класса. Оценивает долю объектов, отнесенных к целевому классу корректно относительно общего числа объектов, отнесенного к целевому классу.

__Recall или же Полнота__ требует выбора целевого класса. Оценивает долю объектов, отнесенных к целевому классу корректно относительно общего числа объектов целевого класса.

__F-score__ – среднее гармоническое между Precision и Recall.

__ROC-AUC__ – площадь под ROC-кривой. Подходит для бинарной классификации. В многоклассовом случае рассматривает каждый класс против всех остальных. Минимальное осмысленное значение $0.5$, значение меньше сигнализирует о том, что банальная смена меток классов на противоположные даст результат выше $0.5$. Для построения информативной кривой требуется модель, которая умеет предсказывать не только метки классов, но и оценивать уверенность в том или ином предсказании. Пример можно увидеть ниже. Почитать подробнее можно [здесь](https://dyakonov.org/2017/07/28/auc-roc-площадь-под-кривой-ошибок/).

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import plot_roc_curve, roc_auc_score
_binary_indices = np.where(dataset[target_column].apply(lambda x: x in ['good', 'bad']))
_binary_problem_feature_matrix = dataset[selected_feature_columns].values[_binary_indices]
_binary_problem_target = dataset[target_column].values[_binary_indices]

lr = LogisticRegression(solver='saga', max_iter=5000)
lr.fit(_binary_problem_feature_matrix, _binary_problem_target)
plot_roc_curve(lr, _binary_problem_feature_matrix, _binary_problem_target)

Как видим, результат выше $0.5$, но при этом далек от идеального. Стоит учесть, что это результат на той выборке, на которой обучалась модель, т.е. указанные классы линейно неразделимы.

Подробное описание основных метрик можно найти [здесь (ru)](https://habr.com/ru/company/ods/blog/328372/) или [здесь (en)](https://scikit-learn.org/stable/modules/model_evaluation.html#classification-metrics).

## Основные модели в задаче классификации
Сущетсвует значительное количество различных моделей. Рассмотрим простые семейства моделей и их свойства.

__Логистическая регрессия__. Одна из наиболее популярных моделей. Строит линейную разделяющую поверхность. При качественном подброе признаков является отличным baseline-решением, с которым достаточно сложно спорить. Нейронная сеть без скрытого слоя – и есть логистическая регрессия.

__Решающее дерево__. Набор простых правил, который делит пространство на непересекающиеся подпространства. Строит нелинейную разделяющую поверхность, крайне склонно к переобучению. Решения деревьев хорошо интерпретируемы, но из-за слабых предсказательных свойств в одиночном виде почти не используются.

__Случайный лес, Random Forest__. Множество решающих деревьев, обученных на случайных подвыборках объектов и случайных признаковых подпространствах. Одна из популярнейших baseline-моделей, с качеством которой достаточно сложно спорить. Одно из лучших решений "из коробки" для первичной проверки гипотез при работе с табличными данными.

__Метод ближайших соседей, kNN__. Метка класса предсказывается на основе ближайших объектов из обучающей выборки. Просто запоминает выборку, слабо подходит для огромных наборов данных без доработок. Страдает от проклятия размерности.

_В данном занятии ограничимся вышеперечисленными моделями. Также стоит выделить следующие модели:_

__Наивный Байесовский классификатор__. Модель, основанная на формуле Байеса. Часто применялась в задаче обнаружения спама. Хорошо работает с категориальными признаками. Может использоваться в качестве простого baseline-решения.

__Градиентный бустинг__. Ансамбль моделей, основанный на бустинге. Решения, основанные на градиентном бустинге, широко применяются на практике.

__Нейронные сети__. Нейронные сети нашли применение во многих областях. Наиболее значимые результаты на данный момент были достигнуты в задачах компьютерного зрения (CV) и обработки естественного языка (NLP).

## Решение задачи классификации.
Разделим выборку на `train` и `test` составляющие. Затем обучим каждую из моделей и проинтерпретируем результаты.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import scikitplot

X_train, X_test, y_train, y_test = train_test_split(
    dataset[selected_feature_columns].values,
    dataset[target_column].values,
    test_size = 0.3,
    random_state = 42
)

# Размеры обучающей и тестовой выборок

print("Shape of X_train :", X_train.shape)
print("Shape of y_train :", y_train.shape)
print("Shape of X_test :", X_test.shape)
print("Shape of y_test :", y_test.shape)

### Логистическая регрессия
Перед обучением отнормируем данные, приведя их к одной шкале.

In [ ]:
# Объединение процедуры масштабирования и самой модели в один pipeline.
pipeline = Pipeline(
    [
        ('scaler', StandardScaler()),
        ('linear', LogisticRegression(multi_class='multinomial', solver='saga', tol=1e-3, max_iter=500))
    ]
)

clf = GridSearchCV(pipeline, param_grid={'linear__C': np.linspace(0.1, 10, 21), 
                                         'linear__penalty': ['l1', 'l2']}, 
                   scoring=('f1_weighted', 'accuracy'), refit='f1_weighted', verbose=1, n_jobs=-1, cv=5)


clf.fit(X_train, y_train)
clf.best_params_, clf.best_score_

In [ ]:
best_lr = clf.best_estimator_

In [ ]:
# evaluating the model
print("Training Accuracy :", clf.best_estimator_.score(X_train, y_train))
print("Testing Accuracy :", clf.best_estimator_.score(X_test, y_test))

# confusion matrix
cm = confusion_matrix(y_test, clf.best_estimator_.predict(X_test))
plt.rcParams['figure.figsize'] = (6, 6)
sns.heatmap(cm ,annot = True)

# classification report
cr = classification_report(y_test, clf.best_estimator_.predict(X_test))
print(cr)

In [ ]:
scikitplot.metrics.plot_roc(y_test, clf.best_estimator_.predict_proba(X_test), figsize=(16, 10))

### Решающее дерево
Нормировка данных не оказывает влияния на решающее дерево, поэтому исключим ее.

In [ ]:
from sklearn.tree import DecisionTreeClassifier

pipeline = Pipeline(
    [
#         ('scaler', StandardScaler()),
        ('tree', DecisionTreeClassifier()),
    ]
)

clf = GridSearchCV(pipeline, param_grid={'tree__max_depth': np.arange(1, 50)}, 
                   scoring=('f1_weighted', 'accuracy'), refit='f1_weighted', verbose=1, n_jobs=-1, cv=5)
clf.fit(X_train, y_train)
clf.best_params_, clf.best_score_

best_tree = clf.best_estimator_

# evaluating the model
print("Training Accuracy :", clf.best_estimator_.score(X_train, y_train))
print("Testing Accuracy :", clf.best_estimator_.score(X_test, y_test))

# confusion matrix
cm = confusion_matrix(y_test, clf.best_estimator_.predict(X_test))
plt.rcParams['figure.figsize'] = (6, 6)
sns.heatmap(cm ,annot = True)

# classification report
cr = classification_report(y_test, clf.best_estimator_.predict(X_test))
print(cr)
scikitplot.metrics.plot_roc(y_test, clf.best_estimator_.predict_proba(X_test), figsize=(16, 10))

Обратите внимание на форму ROC-кривой для одного дерева.

### Random Forest
Нормировка данных не оказывает влияния на решающее дерево, а значит и на их ансамбль. Исключим ее.

In [ ]:
from sklearn.ensemble import RandomForestClassifier

pipeline = Pipeline(
    [
#         ('scaler', StandardScaler()),
        ('forest', RandomForestClassifier()),
    ]
)

clf = GridSearchCV(pipeline, param_grid={'forest__max_depth': np.arange(2, 33, 10),
                                         'forest__n_estimators': np.arange(2, 101, 5)}, 
                   scoring=('f1_weighted', 'accuracy'), refit='f1_weighted', n_jobs=-1, verbose=1, cv=5)
clf.fit(X_train, y_train)

best_rf = clf.best_estimator_

# evaluating the model
print("Training Accuracy :", clf.best_estimator_.score(X_train, y_train))
print("Testing Accuracy :", clf.best_estimator_.score(X_test, y_test))

# confusion matrix
cm = confusion_matrix(y_test, clf.best_estimator_.predict(X_test))
plt.rcParams['figure.figsize'] = (6, 6)
sns.heatmap(cm ,annot = True)

# classification report
cr = classification_report(y_test, clf.best_estimator_.predict(X_test))
print(cr)
scikitplot.metrics.plot_roc(y_test, clf.best_estimator_.predict_proba(X_test), figsize=(16, 10))

### kNN
В случае с kNN нормировка данных важна. В отсутствие нормировки большее внимание будет уделяться признакам в больших шкалах.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

pipeline = Pipeline(
    [
        ('scaler', StandardScaler()),
        ('knn', KNeighborsClassifier()),
    ]
)

clf = GridSearchCV(pipeline, param_grid={'knn__n_neighbors': np.arange(2, 35, 5)}, 
                   scoring=('f1_weighted', 'accuracy'), refit='f1_weighted', n_jobs=-1, verbose=1, cv=5)
clf.fit(X_train, y_train)


best_knn = clf.best_estimator_

# evaluating the model
print("Training Accuracy :", clf.best_estimator_.score(X_train, y_train))
print("Testing Accuracy :", clf.best_estimator_.score(X_test, y_test))

# confusion matrix
cm = confusion_matrix(y_test, clf.best_estimator_.predict(X_test))
plt.rcParams['figure.figsize'] = (6, 6)
sns.heatmap(cm ,annot = True)

# classification report
cr = classification_report(y_test, clf.best_estimator_.predict(X_test))
print(cr)

scikitplot.metrics.plot_roc(y_test, clf.best_estimator_.predict_proba(X_test), figsize=(16, 10))

### Анализ моделей при малых объемах выборки
Также проанализируем качество моделей в зависимости от объема обучающей выборки.

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
import random

from sklearn.metrics import f1_score
colors = ['255,0,0', '0,255,0', '0,0,255', '120,120,0']


estimators = [
    LogisticRegression(C=1.5, penalty='l1', multi_class='multinomial', solver='saga', tol=1e-3, max_iter=1000),
    DecisionTreeClassifier(max_depth=32),
    RandomForestClassifier(max_depth=32, n_estimators=32),
    KNeighborsClassifier(n_neighbors=22)
]
figures = []
for i, clf in enumerate(estimators):
    f1_mean = []
    f1_std = []
    for split in (np.arange(1, 11) / 10):
        f1 = []
        for random_state in [27, 42, 2020, 11, 2]:
            random.seed(random_state)
            index = random.choices(np.arange(X_train_scaled.shape[0]), k=np.ceil(split * X_train_scaled.shape[0]).astype(int))
            clf.fit(X_train_scaled[index], y_train[index])
            y_pred = clf.predict(X_test_scaled)
            f1.append(f1_score(y_test, y_pred, average='weighted'))
        f1_mean.append(np.mean(f1))
        f1_std.append(np.std(f1))
    f1_mean = np.array(f1_mean)
    f1_std = np.array(f1_std)
    figures += [
        go.Scatter(
            x = np.arange(1, 11) / 10,
            y = f1_mean,
            name = f"F1 for {type(clf).__name__}",
            line=dict(color=f'rgb({colors[i]})'),
            fill=None
        ),
        go.Scatter(
            x = np.concatenate([np.arange(1, 11), np.arange(10, -1, -1)]) / 10,
            y = np.concatenate([f1_mean + f1_std, (f1_mean - f1_std)[::-1]]),
            showlegend=False,
            fill='tozerox',
            fillcolor=f'rgba({colors[i]},0.1)',
            mode='none'
        )
    ]
    
fig = go.Figure(figures)
fig.update_xaxes(type='category')
fig.update_layout(
    xaxis_title="Part of the data",
    yaxis_title="score",
    title='Learning curve'
)
fig.show()

Какие выводы можно сделать на основе графика выше? Какая из моделей показывает себя лучше всего? А в условии недостатка данных?

Сформулируйте 1-2 абзаца выводов по аналогии с результатами, полученными на практическом занятии.

__Выводы:__